In [ ]:
import statsmodels  
statsmodels.__version__
#pip install statsmodels --upgrade --user (if version is older of statsmodel.)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 5 Differernt models for time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA

from sklearn.metrics import mean_absolute_error

In [ ]:
data=pd.read_csv(r'ads.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data=pd.read_csv('ads.csv', parse_dates=['Time']) #for converting into datetime64[ns]

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data=pd.read_csv('ads.csv',parse_dates=['Time'],index_col=['Time'])
#to set the data and time as index

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.index.sort_values()  
#command for sorting the index time and date 

In [ ]:
#data=data.Ads.sort_values() #to sort by any column of the dataset

In [ ]:
#data.head()

# Sort By Index

In [ ]:
data=data.sort_index()

In [ ]:
data.head()

# Get the frequency of the Time/Date Data

In [ ]:
pd.infer_freq(data.index) 
#to calculate the freq of data/time column i.e index column in our case

Preprocessing done

# Visualize the data

In [ ]:
fig=go.Figure(go.Scatter(x=data.index, y=data.Ads,mode='lines'))
fig.show()

In [ ]:
data.Ads

In [ ]:
data.index

# Decompose the above chart using matplotlib

In [ ]:
decomposed_data=seasonal_decompose(data,model='additive')
decomposed_data

In [ ]:
type(decomposed_data)

In [ ]:
plt.rcParams['figure.figsize']=[20,10]
plt.rcParams.update({'font.size':20})
 
decomposed_data.plot()
plt.show()

# Multiplicative Method

In [ ]:
decomposed_data=seasonal_decompose(data,model='multiplicative')


In [ ]:
plt.rcParams['figure.figsize']=[20,10]
plt.rcParams.update({'font.size':20})
 
    
decomposed_data.plot()
plt.show()

# Calculate the Moving Average

In [ ]:
i=12
data['MA']=data.rolling(i).mean()
#calculating moving avg for smooth graph

data.head(i+1)

In [ ]:
mae=mean_absolute_error(data.Ads[i:],data.MA[i:])
round(mae,2)

print("Mean Absolute Error:{}".format(round(mae,2)))

In [ ]:
fig=go.Figure(go.Scatter(x=data.index, y=data.Ads,mode='lines'))

fig.add_trace(go.Scatter(x=data.index,y=data.MA,mode='lines'))
fig.update_layout(title="Mean_Absolute_Error:{}".format(round(mae,2)))
fig.show()

## Calculating Mean Absolute percentage error & Accuracy

In [ ]:
def mean_absolute_percentage_error(actual,predicted):
    return round(np.mean(np.abs(100*(actual-predicted)/actual)),2)

In [ ]:
mape=mean_absolute_percentage_error(data.Ads[i:],data.MA[i:])
mape

In [ ]:
Accuracy=100-mape
Accuracy

# Split data into test train

In [ ]:
split=int(len(data)*0.7) ### 0.7 for 70% data----cahnge as per requirement
#data.index.freq= 'H' either this ot next comand use fot freq
data.index.freq=pd.infer_freq(data.index)
train = data.iloc[:split,0] #151 lines in train
test = data.iloc[split:,0]


1.AR -- Auto Regression-- it considers that current value is dependent on previous n values

2.Moving Avg: Uses last n values to calculate rolling avg and uses it for the prediction

3.ARMA: Is the model developed to apply both AR and MA Methoda,we can specify n values as 0 if we dont want to use either of them.

In [ ]:
train

In [ ]:
test

# Moving Average

###  Step 1. Train Model

In [ ]:
model = ARMA(train,order = (0,5),freq='H')# 0 for AR and 5 for MA
model_fit=model.fit()

### Step 2.Predict Result

In [ ]:
pred=model_fit.predict(start=test.index[0],end=test.index[-1])
pred

### Step 3. Error calculation

In [ ]:
MAPE=mean_absolute_percentage_error(test,pred)
print('Mean Absolute Percentage Error :{}'.format(MAPE))
print('Accuracy:{}'.format(100-MAPE))

### Step 4.Plot the Predicted values

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index,y=train, mode='lines',name='Train'))
fig.add_trace(go.Scatter(x=test.index,y=test, mode='lines',name='Test'))
fig.add_trace(go.Scatter(x=pred.index,y=pred, mode='lines',name='Predict'))
fig.update_layout(title='Accuracy:{}%'.format(100-MAPE))
fig.show()

# AutoRegression

In [ ]:
#step1
model=ARMA(train,order=(15,0),freq='H') #AR at degree 12 refer phone  AR Pic
#RA=2 and MA=0
model_fit=model.fit()

#step2 & step 3
# Prediction and accuracy calculation
pred=model_fit.predict(start=test.index[0],end=test.index[-1])
MAPE=mean_absolute_percentage_error(test,pred)

#step4
#plotting the figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index,y=train, mode='lines',name='Train'))
fig.add_trace(go.Scatter(x=test.index,y=test, mode='lines',name='Test'))
fig.add_trace(go.Scatter(x=pred.index,y=pred, mode='lines',name='Predict'))
fig.update_layout(title='Accuracy:{}%'.format(100-MAPE))
fig.show()

# Auto Regressive Moving Average

In [ ]:
#step1
model=ARMA(train,order=(15,6),freq='H')  
#AR=15 and MA=6
model_fit=model.fit()

#step2 & step 3
# Prediction and accuracy calculation
pred=model_fit.predict(start=test.index[0],end=test.index[-1])
MAPE=mean_absolute_percentage_error(test,pred)

#step4
#plotting the figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index,y=train, mode='lines',name='Train'))
fig.add_trace(go.Scatter(x=test.index,y=test, mode='lines',name='Test'))
fig.add_trace(go.Scatter(x=pred.index,y=pred, mode='lines',name='Predict'))
fig.update_layout(title='Accuracy:{}%'.format(100-MAPE))
fig.show()

# ARIMA-- Auto Regression Integrated Moving Average

In [ ]:
#step1
model=ARIMA(train,order=(12,1,6),freq='H')  
#AR=2 I=1 MA=2
model_fit=model.fit(disp = False)

#step2 & step 3
# Prediction and accuracy calculation
pred=model_fit.predict(start=test.index[0],end=test.index[-1],typ='levels')
MAPE=mean_absolute_percentage_error(test,pred)

#step4
#plotting the figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index,y=train, mode='lines',name='Train'))
fig.add_trace(go.Scatter(x=test.index,y=test, mode='lines',name='Test'))
fig.add_trace(go.Scatter(x=pred.index,y=pred, mode='lines',name='Predict'))
fig.update_layout(title='Accuracy:{}%'.format(100-MAPE))
fig.show()